In [ ]:
# Data should be in format
# index,enerygy,tempF
# 2012-01-01 00:00:00+00:00,30.2197,49.9725
# load the data as data=pd.read_csv('data.csv', index_col=0, parse_dates=True)
# 

# Follow lines do notthing
#from eemeter.modeling.models import caltrack
#model = CaltrackDailyModel(fit_cdd = True, grid_search = True)
#output = model.fit(mydata)

In [39]:
# Loading the NATGrid hourly data
# 
import json
import numpy as np
import pytz
import pandas 
from eemeter.weather.location import zipcode_to_usaf_station
from eemeter.weather import ISDWeatherSource


file_name = '/vagrant/etl-natgrid-lime/test_data_hourly/raw_site_data/2017-07-27/27871.json'
def prepare_hrl_data_md(fl):
    with open(fl) as fp:
        js_data = json.load(fp)
    
    dt = js_data['data']
    #print "Keys in the Json data ",js_data.keys()
    result = []
    index = []
    energy_consumed= []
    for energy_data in dt:
        # energy_cs can have none values
        energy_cs = energy_data['energy_consumed']
        time_st = energy_data['t_stamp']
        index.append(np.datetime64(time_st, 'h'))
        energy_consumed.append(energy_cs)
    #print "t_step (I don't know what that means" , js_data['t_step']
    dt_index = pandas.DatetimeIndex(index, dtype='datetime64[ns]', freq='H',tz=pytz.UTC)
    df = pandas.DataFrame({'energy' : np.array(energy_consumed, dtype=np.float16)}, index=dt_index)
    return df
rs = prepare_hrl_data_md(file_name)
#print rs

usaf = zipcode_to_usaf_station('27871')
weather_source = ISDWeatherSource(usaf)

#temps = weather_source.indexed_temperatures(index, "degF")

# date_range creates numpy datetime64 https://docs.scipy.org/doc/numpy/reference/arrays.datetime.html
#daily_index = pd.date_range('2017-07-01', periods=24,freq='H', tz=pytz.UTC)
#print daily_index
tt = weather_source.indexed_temperatures(rs.index, "degF")
rs = rs.assign(temp=tt.values)
print  rs[1000:2000]

                            energy  temp
2017-07-07 16:00:00+00:00   566.50  89.6
2017-07-07 17:00:00+00:00   631.00  89.6
2017-07-07 18:00:00+00:00   377.75  91.4
2017-07-07 19:00:00+00:00   489.00  86.0
2017-07-07 20:00:00+00:00   448.25  89.6
2017-07-07 21:00:00+00:00   532.00  91.4
2017-07-07 22:00:00+00:00   555.00  89.6
2017-07-07 23:00:00+00:00   571.50  89.6
2017-07-08 00:00:00+00:00   550.00  82.4
2017-07-08 01:00:00+00:00   584.00  80.6
2017-07-08 02:00:00+00:00   555.00  80.6
2017-07-08 03:00:00+00:00   543.00  80.6
2017-07-08 04:00:00+00:00   574.50  78.8
2017-07-08 05:00:00+00:00   516.00  80.6
2017-07-08 06:00:00+00:00   395.75  78.8
2017-07-08 07:00:00+00:00   369.50  77.0
2017-07-08 08:00:00+00:00   396.25  77.0
2017-07-08 09:00:00+00:00   381.25  77.0
2017-07-08 10:00:00+00:00   378.50  77.0
2017-07-08 11:00:00+00:00   397.25  78.8
2017-07-08 12:00:00+00:00   352.00  80.6
2017-07-08 13:00:00+00:00   413.25  82.4
2017-07-08 14:00:00+00:00   417.25  86.0
2017-07-08 15:00